In [1]:
import pandas as pd 
import numpy as np
import datetime
import Analysis
import pandas_ta as ta

In [2]:
df = pd.read_csv('final_df.csv')

In [3]:
df['Fetch_Time'] = pd.to_datetime(df['Fetch_Time'])

In [4]:
strike = df.Strike_Price.unique()

In [26]:
combination = df[['Strike_Price','Expiry_Date']].drop_duplicates()

In [23]:
df[df['Strike_Price'] == strike[5]]['Expiry_Date'].unique()

array(['2025-06-26', '2025-09-25', '2025-12-24', '2026-06-25',
       '2026-12-31', '2027-06-24', '2027-12-30', '2028-06-29',
       '2028-12-28', '2029-06-28', '2029-12-27', '2026-03-26'],
      dtype=object)

In [5]:
new = df[(df['Strike_Price'] == 24550) & (df['Expiry_Date']=='2025-05-08')]

In [ ]:
new['Date'] = new['Fetch_Time'].dt.date
chk = new[new['Date'] == datetime.date.today()][['Fetch_Time','CALLS_LTP']]
new[new['Date'] == datetime.date.today()]
chk1 = chk.resample('5min',on='Fetch_Time').agg({'CALLS_LTP':['first', 'min', 'max', 'last']}).reset_index()
chk1.columns = ['datetime', 'Open', 'Low', 'High', 'Close']
chk1['smma'] = Analysis.calculate_smma(chk1, period=7)
chk1['rsi'] = ta.rsi(chk1['Close'])
chk1['dema'] = ta.dema(chk1['Close'], length=10)
chk1['super_trend'] = ta.supertrend(chk1['High'], chk1['Low'], chk1['Close'], length=10, multiplier=3)['SUPERT_10_3.0']
chk1['super_trend_color'] = np.where(chk1['Close'] > chk1['super_trend'] , "green" , "red")
chk1['BBM'] = ta.bbands(close = chk1['Close'], length=20, std=2)['BBM_20_2.0']
chk1['changeover'] = np.insert(np.diff(np.where(chk1['super_trend_color'] == 'green' , 1 , 0)),0,0)
chk1['condition'] = np.insert(np.diff(np.where((chk1['BBM'] < chk1['smma']) & (chk1['smma'] < chk1['dema']) ,1,0)),0,0)
chk1 = chk1.iloc[np.where(((chk1['changeover'] == 1) & (chk1['condition'] == 1)))] 

/tmp/ipykernel_1569/788553029.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new['Date'] = new['Fetch_Time'].dt.date


In [18]:
chk1

,datetime,Open,Low,High,Close,smma,rsi,dema,super_trend,super_trend_color,BBM,changeover,condition


In [37]:
dff = []
for i, row in combination.iterrows():
   try:
      strike = row['Strike_Price']
      expiry = row['Expiry_Date']
      new = df[(df['Strike_Price'] == strike) & (df['Expiry_Date']== expiry)].copy()
      new['Date'] = new['Fetch_Time'].dt.date
      chk = new[new['Date'] == datetime.date.today()][['Fetch_Time','CALLS_LTP']]
      new[new['Date'] == datetime.date.today()]
      chk1 = chk.resample('5min',on='Fetch_Time').agg({'CALLS_LTP':['first', 'min', 'max', 'last']}).reset_index()
      chk1.columns = ['datetime', 'Open', 'Low', 'High', 'Close']
      chk1['smma'] = Analysis.calculate_smma(chk1, period=7)
      chk1['rsi'] = ta.rsi(chk1['Close'])
      chk1['dema'] = ta.dema(chk1['Close'], length=10)
      chk1['super_trend'] = ta.supertrend(chk1['High'], chk1['Low'], chk1['Close'], length=10, multiplier=3)['SUPERT_10_3.0']
      chk1['super_trend_color'] = np.where(chk1['Close'] > chk1['super_trend'] , "green" , "red")
      chk1['BBM'] = ta.bbands(close = chk1['Close'], length=20, std=2)['BBM_20_2.0']
      chk1['changeover'] = np.insert(np.diff(np.where(chk1['super_trend_color'] == 'green' , 1 , 0)),0,0)
      chk1['condition'] = np.insert(np.diff(np.where((chk1['BBM'] < chk1['smma']) & (chk1['smma'] < chk1['dema']) ,1,0)),0,0)
      chk1 = chk1.iloc[np.where(((chk1['changeover'] == 1) & (chk1['condition'] == 1)))] 
      chk1['strike'] = strike
      chk1['expiry'] = expiry
      dff.append(chk1)
   except:
      pass

In [38]:
pd.concat(dff)

,datetime,Open,Low,High,Close,smma,rsi,dema,super_trend,super_trend_color,BBM,changeover,condition,strike,expiry
53,2025-05-02 13:40:00,3500.00,3500.00,3595.70,3595.70,3521.272375,64.239571,3523.418461,3504.678166,green,3514.9575,1,1,21000,2025-06-26
58,2025-05-02 14:05:00,1911.30,1911.30,1911.30,1911.30,1779.500595,79.276040,1799.139012,1856.941699,green,1760.8925,1,1,22500,2025-05-08
60,2025-05-02 14:15:00,1352.70,1352.70,1490.10,1490.10,1376.191711,81.364437,1393.971898,1374.269525,green,1359.5700,1,1,22900,2025-05-08
57,2025-05-02 14:00:00,1643.35,1643.35,1643.35,1643.35,1555.247594,66.662700,1556.756837,1597.944550,green,1543.8025,1,1,22900,2025-05-29
51,2025-05-02 13:30:00,1670.00,1670.00,1700.00,1700.00,1622.251564,79.949437,1632.713549,1638.127254,green,1620.7500,1,1,23100,2025-06-26
51,2025-05-02 13:30:00,1265.00,1265.00,1306.30,1306.30,1191.599255,71.320440,1198.519512,1218.974857,green,1187.2625,1,1,23200,2025-05-15
59,2025-05-02 14:10:00,1077.45,1077.45,1135.05,1135.05,1090.452023,63.826341,1092.216075,1084.024273,green,1086.1700,1,1,23250,2025-05-08
55,2025-05-02 13:50:00,1260.00,1260.00,1349.95,1346.75,1278.145024,74.429959,1282.791557,1272.905510,green,1268.8375,1,1,23250,2025-05-29
50,2025-05-02 13:25:00,1320.10,1320.10,1332.00,1332.00,1288.594699,56.854853,1295.610688,1275.360748,green,1281.7300,1,1,23500,2025-06-26
52,2025-05-02 13:35:00,717.00,717.00,717.00,717.00,643.525283,76.700308,657.788991,670.398169,green,642.5250,1,1,23850,2025-05-15
